In [ ]:
import sys
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
SRC_PATH = '/content/gdrive/MyDrive/MP FEB/Colab'
sys.path.append(SRC_PATH)

!pip install wandb -qqq
import wandb
wandb.login()

wandb.init(
    # Set the project where this run will be logged
    project="DDHT_poc", 
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    #name="experiment 1"
    # Track hyperparameters and run metadata
    #config={
      #"learning_rate": 0.02,
      #"architecture": "CNN",
      #"dataset": "CIFAR-100",
      #"epochs": 10,}
    )

In [ ]:
import numpy as np
import pandas as pd
import torch
import copy

import matplotlib.pyplot as plt
%matplotlib inline

DEVICE = 'cuda'

# 1. Import Dataset

In [ ]:
from torch.utils.data import DataLoader
from poc_data import PocDataset, display_sample


NUM_CASES = 2**6
BATCH_SIZE = 2**4

poc_raw_dataset = PocDataset(num_cases=NUM_CASES, augment=True)
data_loader = DataLoader(dataset=poc_raw_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        pin_memory=True)

batch_data, batch_data_length, batch_event, batch_tte, _ = next(iter(data_loader))

print(batch_data.shape)
print(batch_data_length.shape)
print(batch_event.shape)

### Data Exploration

In [ ]:
#Take a look at the longitudional data
display_sample(batch_data[0], batch_data_length[0], batch_event[0])
display_sample(batch_data[1], batch_data_length[1], batch_event[1])
display_sample(batch_data[2], batch_data_length[2], batch_event[2])
display_sample(batch_data[3], batch_data_length[3], batch_event[3])

### Test Sample

In [ ]:
test_poc_raw_dataset = PocDataset(num_cases=1)
test_data_loader = DataLoader(dataset=test_poc_raw_dataset,batch_size=1,pin_memory=True)
test_batch_data, test_batch_data_length, test_batch_event, test_tte, test_meta = next(iter(test_data_loader))
display_sample(test_batch_data[0], test_batch_data_length[0], test_batch_event[0])

# 2. Hyperparameters

In [ ]:
from torch.nn import MSELoss
from tqdm import trange, tqdm

from torch.optim import Adam

from dynamicDeepHit import EncoderRNN, AttnDecoderRNN, CauseSpecificSubnetwork, DynamicDeepHit
from losses import loss_1_batch, loss_2_batch, loss_3_batch

NUM_EPOCHS = 1

LEARNING_RATE_ENCODER = 0.001
LEARNING_RATE_DECODER = 0.001
LEARNING_RATE_CAUSESS = 0.0005

LOSS_1_AMPLIFIER = 1
LOSS_2_AMPLIFIER = 1
LOSS_3_AMPLIFIER = 1

RUN_VALIDATION_ROUND = False
VAL_NUM_CASES_RUNTIME = BATCH_SIZE

input_size = 5
output_size = input_size
MAX_LENGTH = 2*12
NUM_CAUSES = 3
hidden_size_encoder = 256
hidden_size_attention = 512
fc_size_encoder = 512
SIGMA = 0.1

# 3. Defining The Model

In [ ]:
# initialize model
encoder = EncoderRNN(input_size, hidden_size_encoder, fc_size_encoder).to(DEVICE)
decoder = AttnDecoderRNN(hidden_size_encoder, hidden_size_attention, output_size).to(DEVICE)
causess = CauseSpecificSubnetwork(hidden_size_encoder, input_size, MAX_LENGTH, NUM_CAUSES).to(DEVICE)
DDHT = DynamicDeepHit(encoder, decoder, causess, MAX_LENGTH, DEVICE)

# intialize optimizer
optimizer_encoder = Adam(encoder.parameters(), lr=LEARNING_RATE_ENCODER)
optimizer_decoder = Adam(decoder.parameters(), lr=LEARNING_RATE_DECODER)
optimizer_causess = Adam(causess.parameters(), lr=LEARNING_RATE_CAUSESS)

# 4. Training the model

In [ ]:
PATH = "/content/gdrive/MyDrive/MP FEB/Colab/models/model_v14_2048.pth"

# start training
for epoch in trange(NUM_EPOCHS):

  epoch_loss = 0

  for batch_number, data in enumerate(data_loader):

    batch_loss = 0

    optimizer_encoder.zero_grad()
    optimizer_decoder.zero_grad()
    optimizer_causess.zero_grad()

    batch_data, batch_data_length, batch_event, batch_tte, _ = data
    batch_data = batch_data.to(DEVICE)
    batch_data_length = batch_data_length.to(DEVICE)
    batch_event = batch_event.to(DEVICE)
    batch_tte = batch_tte.to(DEVICE)
    
    output_batch, first_hitting_time_batch = DDHT(batch_data, batch_data_length)

    loss1 = LOSS_1_AMPLIFIER*loss_1_batch(first_hitting_time_batch, batch_event, batch_tte, batch_data_length, MAX_LENGTH, DEVICE)
    loss2 = LOSS_2_AMPLIFIER*loss_2_batch(first_hitting_time_batch, batch_event, batch_tte, batch_data_length, NUM_CAUSES, MAX_LENGTH, SIGMA, DEVICE)
    loss3 = LOSS_3_AMPLIFIER*loss_3_batch(output_batch, batch_data.detach())

    batch_loss = loss1 + loss2 + loss3
    batch_loss.backward()

    epoch_loss += batch_loss.detach()

    wandb.log({"train_loss1": loss1.item(), "train_loss2": loss2.item(), "train_loss3": loss3.item()})

    optimizer_encoder.step()
    optimizer_decoder.step()
    optimizer_causess.step()

    if batch_number % 2**8 == 0:
      torch.save(DDHT.state_dict(), PATH)


  if RUN_VALIDATION_ROUND:
    # validating round
    DDHT.eval()

    with torch.no_grad():
      val_poc_raw_dataset = PocDataset(num_cases=VAL_NUM_CASES_RUNTIME)
      val_data_loader = torch.utils.data.DataLoader(val_poc_raw_dataset,batch_size=VAL_NUM_CASES_RUNTIME)
      val_batch_data, val_data_length, val_batch_event, val_batch_tte, _ = next(iter(val_data_loader))
      val_batch_data = val_batch_data.to(DEVICE)
      val_data_length = val_data_length.to(DEVICE)
      val_batch_event = val_batch_event.to(DEVICE)
      val_batch_tte = val_batch_tte.to(DEVICE)

      val_output_batch, val_first_hitting_time_batch = DDHT(val_batch_data, val_data_length)

      val_loss1 = LOSS_1_AMPLIFIER*loss_1_batch(val_first_hitting_time_batch, val_batch_event, val_batch_tte, val_batch_data_length, MAX_LENGTH, DEVICE)/VAL_NUM_CASES_RUNTIME
      val_loss2 = LOSS_2_AMPLIFIER*loss_2_batch(val_first_hitting_time_batch, val_batch_event, val_batch_tte, val_batch_data_length, NUM_CAUSES, MAX_LENGTH, SIGMA, DEVICE)/VAL_NUM_CASES_RUNTIME
      val_loss3 = LOSS_3_AMPLIFIER*loss_3_batch(val_output_batch, val_batch_data.detach())/VAL_NUM_CASES_RUNTIME

      wandb.log({"val_loss1": val_loss1.item(), "val_loss2": val_loss2.item(), "val_loss3": val_loss3.item()})
      wandb.log({"train_epoch_loss" : epoch_loss.item(), "val_epoch_loss" : val_loss1.item() + val_loss2.item() + val_loss3.item(),"epoch": epoch})

    DDHT.train()
    # end validating round

  torch.save(DDHT.state_dict(), PATH)

wandb.finish() 

# 4. Testing the Model

In [ ]:
PATH = "C:/Users/marij/Desktop/THESIS ECO/DDHT/DDHT_pytorch/models/model_v14_2048.pth"
#torch.save(DDHT.state_dict(), PATH)
DDHT.load_state_dict(torch.load(PATH))

### Scoring our model - sampling a new test set

In [ ]:
from utils import plot_fht_and_cif
from losses import CIF_K_tau

DDHT.eval()
test_data_loader = PocDataset(num_cases=1, test_set=True, repays=False)
test_batch_data, test_batch_data_length, test_batch_event, test_batch_tte, test_meta = next(iter(test_data_loader))
test_batch_data = test_batch_data.unsqueeze(0).to(DEVICE)
test_batch_data_length = test_batch_data_length.unsqueeze(0).to(DEVICE)
test_batch_event = test_batch_event.unsqueeze(0).to(DEVICE)

test_output, test_first_hitting_time = DDHT(test_batch_data, test_batch_data_length)
print("sample has length %d" % test_batch_data_length[0])
print("sample will experience event %d at time %d, but shows event %d" % (test_meta['ground_truth_event'], test_meta['age'], test_batch_event[0]))


test_first_hitting_time_argmax = test_first_hitting_time.argmax().item()
model_event_prediction = test_first_hitting_time_argmax // MAX_LENGTH
model_tte_prediction = test_first_hitting_time_argmax % MAX_LENGTH
print("the model predicts the event %d at time %d" % (model_event_prediction, model_tte_prediction + 1))

print("probability of prepay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 0, 24, test_batch_data_length[0], MAX_LENGTH).item())
print("probability of default event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 1, 24, test_batch_data_length[0], MAX_LENGTH).item())
print("probability of full repay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 2, 24, test_batch_data_length[0], MAX_LENGTH).item())

plot_fht_and_cif(test_first_hitting_time[0], test_batch_data_length[0], MAX_LENGTH)

In [ ]:
test_batch_data, test_batch_data_length, test_batch_event, test_batch_tte, test_meta = next(iter(test_data_loader))
test_batch_data_length = test_batch_data_length.unsqueeze(0).to(DEVICE)
test_batch_event = test_batch_event.unsqueeze(0).to(DEVICE)

masked_data = copy.deepcopy(test_batch_data.unsqueeze(0))

#masked_data[:,:,0] = 0
#masked_data[:,:,1] = 0
#masked_data[:,:,2] = 0
#masked_data[:,:,3] = 0
#masked_data[:,:,4] = 0

masked_data = masked_data.to(DEVICE)

test_output, test_first_hitting_time = DDHT(masked_data, test_batch_data_length)
print("sample has length %d" % test_batch_data_length[0])
print("sample will experience event %d at time %d, but shows event %d" % (test_meta['ground_truth_event'], test_meta['age'], test_batch_event[0]))


test_first_hitting_time_argmax = test_first_hitting_time.argmax().item()
model_event_prediction = test_first_hitting_time_argmax // MAX_LENGTH
model_tte_prediction = test_first_hitting_time_argmax % MAX_LENGTH
print("the model predicts the event %d at time %d" % (model_event_prediction, model_tte_prediction + 1))

print("probability of prepay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 0, 24, test_batch_data_length[0], MAX_LENGTH).item())
print("probability of default event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 1, 24, test_batch_data_length[0], MAX_LENGTH).item())
print("probability of full repay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 2, 24, test_batch_data_length[0], MAX_LENGTH).item())

plot_fht_and_cif(test_first_hitting_time[0], test_batch_data_length[0], MAX_LENGTH)

In [ ]:
#Probability that this loan will prepay, default or repay in the comming "delta" months?

delta = 6  #This is variable

evaluation_time = min(int(test_batch_data_length[0].item()) + delta, MAX_LENGTH)
print("In the comming %d months, the probability that a specific event happens is:" % delta)

p_ev0 = CIF_K_tau(test_first_hitting_time[0], 0, evaluation_time, test_batch_data_length[0], MAX_LENGTH).item()
p_ev1 = CIF_K_tau(test_first_hitting_time[0], 1, evaluation_time, test_batch_data_length[0], MAX_LENGTH).item()
p_ev2 = CIF_K_tau(test_first_hitting_time[0], 2, evaluation_time, test_batch_data_length[0], MAX_LENGTH).item()

print("probability a prepay happens = %.3f" % p_ev0)
print("probability a default happens = %.3f" % p_ev1)
print("probability a full repay happens = %.3f" % p_ev2)

sum = p_ev0 + p_ev1 + p_ev2
print("The probability anything happens = %.3f" % sum)


In [ ]:
#Probability that this loan will prepay, default or repay with less data
test_batch_data, test_batch_data_length, test_batch_event, test_batch_tte, test_meta = next(iter(test_data_loader))

delta = 5 #This is variable
shortened_length = test_batch_data_length - delta
shortened_test_batch_data = copy.deepcopy(test_batch_data)
shortened_test_batch_data[shortened_length:test_batch_data_length] = torch.zeros(delta, input_size)
shortened_test_batch_data_length = copy.deepcopy(shortened_length)

shortened_test_batch_data = shortened_test_batch_data.unsqueeze(0).to(DEVICE)
shortened_test_batch_data_length = shortened_test_batch_data_length.unsqueeze(0).to(DEVICE)
test_batch_event = test_batch_event.unsqueeze(0).to(DEVICE)

test_output, test_first_hitting_time = DDHT(shortened_test_batch_data, shortened_test_batch_data_length)
print("sample has length %d, but we concatenated to %d" % (test_batch_data_length, shortened_test_batch_data_length[0]))
print("sample will experience event %d at time %d, but shows event %d" % (test_meta['ground_truth_event'], test_meta['age'], test_batch_event[0]))


test_first_hitting_time_argmax = test_first_hitting_time.argmax().item()
model_event_prediction = test_first_hitting_time_argmax // MAX_LENGTH
model_tte_prediction = test_first_hitting_time_argmax % MAX_LENGTH
print("the model predicts the event %d at time %d" % (model_event_prediction, model_tte_prediction + 1))

print("probability of prepay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 0, 24, shortened_test_batch_data_length[0], MAX_LENGTH).item())
print("probability of default event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 1, 24, shortened_test_batch_data_length[0], MAX_LENGTH).item())
print("probability of full repay event = %.2f" % CIF_K_tau(test_first_hitting_time[0], 2, 24, shortened_test_batch_data_length[0], MAX_LENGTH).item())

plot_fht_and_cif(test_first_hitting_time[0], shortened_test_batch_data_length[0], MAX_LENGTH)

In [ ]:
from utils import plot_gamma

SENSITIVITY_BATCH_SIZE = 2**8

sensitivity_poc_dataset = PocDataset(num_cases=SENSITIVITY_BATCH_SIZE, test_set=True, repays=True, augment=False)
sensitivity_data_loader = DataLoader(dataset=sensitivity_poc_dataset, batch_size=SENSITIVITY_BATCH_SIZE, shuffle=True, pin_memory=True)

delta = 6 #This is variable

#do 2 placeholders for the results
min_gamma = torch.zeros(NUM_CAUSES, input_size)
max_gamma = torch.zeros(NUM_CAUSES, input_size)

for covariate_index in range(input_size):
    #Probability that this loan will prepay, default or repay with less data
    sensitivity_batch_data, sensitivity_batch_data_length, sensitivity_batch_event, sensitivity_batch_tte, sensitivity_meta = next(iter(sensitivity_data_loader))

    #assuming it's minimum & maximum from the whole batch, not from a single sample
    batch_min = torch.tensor(float('Inf'))
    batch_max = (-1)*torch.tensor(float('Inf'))

    #we have to iterate over it otherwise, we catch the zero's that are meant as NANs in our case
    for sample, data_length in zip(sensitivity_batch_data, sensitivity_batch_data_length):
        sample_min = torch.min(sample[:data_length,covariate_index])
        sample_max = torch.max(sample[:data_length,covariate_index])

        if sample_min < batch_min:
            batch_min = sample_min

        if sample_max > batch_max:
            batch_max = sample_max

    min_sensitivity_batch_data = copy.deepcopy(sensitivity_batch_data)
    max_sensitivity_batch_data = copy.deepcopy(sensitivity_batch_data)

    #for safety we iterate again, since we otherwise fill in the zero's that are meant as NANs
    for sample_index, data_length in enumerate(sensitivity_batch_data_length):
        min_sensitivity_batch_data[sample_index,:data_length, covariate_index] = batch_min
        max_sensitivity_batch_data[sample_index,:data_length, covariate_index] = batch_max

    min_sensitivity_batch_data = min_sensitivity_batch_data.to(DEVICE)
    max_sensitivity_batch_data = max_sensitivity_batch_data.to(DEVICE)
    sensitivity_batch_data_length = sensitivity_batch_data_length.to(DEVICE)

    #for safety we iterate again, because the previous iteration might be absorbed by the one above that
    for sample_index, data_length in enumerate(sensitivity_batch_data_length):
        for cause_index in range(NUM_CAUSES):
            evaluation_time = min(int(data_length.item()) + delta, MAX_LENGTH)

            _, max_fht = DDHT(max_sensitivity_batch_data[sample_index].unsqueeze(0), data_length)
            _, min_fht = DDHT(min_sensitivity_batch_data[sample_index].unsqueeze(0), data_length)

            max_gamma[cause_index, covariate_index] += CIF_K_tau(max_fht[0], cause_index, evaluation_time, data_length, MAX_LENGTH).item()
            min_gamma[cause_index, covariate_index] += CIF_K_tau(min_fht[0], cause_index, evaluation_time, data_length, MAX_LENGTH).item()


gamma = (1/SENSITIVITY_BATCH_SIZE)*(min_gamma - max_gamma)
plot_gamma(gamma)